In [ ]:
import os.path as osp
import subprocess
import os
from configs.constants import XMAGICAL_EMBODIMENT_TO_ENV_NAME

from absl import app
from absl import logging
from configs.constants import ALGORITHMS
from configs.constants import EMBODIMENTS
from torchkit.experiment import string_from_kwargs
from torchkit.experiment import unique_id
import yaml
import random


In [ ]:
seeds = [0, 5]
device = "cuda:0"

experiment_name = '/home/user/xirl/exp/exp864/'
embodiment = 'longstick'

with open(osp.join(experiment_name, "config.yaml"), "r") as fp:
    kwargs = yaml.load(fp, Loader=yaml.FullLoader)

if kwargs["algorithm"] == "goal_classifier":
    reward_type = "goal_classifier"
else:
    reward_type = "distance_to_goal"

# Map the embodiment to the x-MAGICAL env name.
env_name = XMAGICAL_EMBODIMENT_TO_ENV_NAME[embodiment]

logging.info("Experiment name: %s", experiment_name)

In [ ]:
# Execute each seed in parallel.
procs = []
for seed in range(*list(map(int, seeds))):
    procs.append(
        subprocess.Popen([  # pylint: disable=consider-using-with
            "python",
            "train_policy.py",
            "--experiment_name",
            experiment_name,
            "--env_name",
            f"{env_name}",
            "--config",
            f"configs/xmagical/rl/env_reward.py:{embodiment}",
            "--config.reward_wrapper.pretrained_path",
            f"{experiment_name}",
            "--config.reward_wrapper.type",
            f"{reward_type}",
            "--seed",
            f"{seed}",
            "--device",
            f"{device}",
            '--resume',
        ]))

# Wait for each seed to terminate.
for p in procs:
    p.wait()